In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. Install dan Import Library
!pip install -q transformers datasets sastrawi
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from datasets import Dataset
import matplotlib.pyplot as plt
import seaborn as sns


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.4 MB/s eta 0:00:00


In [3]:
# 2. Load Dataset
df = pd.read_csv("/content/drive/MyDrive/uas/product_reviews_dirty.csv")
# Use the correct column names 'text' and 'rating'
df = df[['text', 'rating']].dropna()

# Label Encoding
# Map rating to 1 for ratings >= 4 and 0 for ratings <= 2
df['sentiment'] = df['rating'].apply(lambda x: 1 if x >= 4 else (0 if x <= 2 else None))
# Drop rows where sentiment is None (rating is 3)
df = df.dropna(subset=['sentiment'])

# Convert sentiment to integer type
df['sentiment'] = df['sentiment'].astype(int)

# Rename 'text' column to 'Customer_Review' for consistency with the next cell if needed,
# but the next cell uses 'Customer_Review' which might be a remnant from a different script.
# Let's keep 'text' for now as it's used in preprocessing.
# df.rename(columns={'text': 'Customer_Review'}, inplace=True)

# Rename 'sentiment' column to 'Sentiment' for consistency if needed
# df.rename(columns={'sentiment': 'Sentiment'}, inplace=True)

In [4]:
# 3. Preprocessing (Lowercase, Cleaning, Stemming)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)  # hapus URL
    text = re.sub(r"[^\w\s]", "", text)  # hapus tanda baca
    text = re.sub(r"\d+", "", text)      # hapus angka
    text = text.strip()
    return stemmer.stem(text)

df['cleaned'] = df['text'].astype(str).apply(clean_text)

In [6]:
# 4. Tokenization untuk LSTM dan GRU
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['cleaned'])
sequences = tokenizer.texts_to_sequences(df['cleaned'])
padded = pad_sequences(sequences, padding='post', maxlen=100)

X = padded
y = df['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# 5. Model LSTM
def build_lstm():
    model = Sequential([
        Embedding(10000, 64, input_length=100),
        Bidirectional(LSTM(64)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

lstm_model = build_lstm()
lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9767 - loss: 0.1267 - val_accuracy: 0.9817 - val_loss: 0.0591
Epoch 2/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9831 - loss: 0.0497 - val_accuracy: 0.9809 - val_loss: 0.0565
Epoch 3/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9889 - loss: 0.0323 - val_accuracy: 0.9790 - val_loss: 0.0627
Epoch 4/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9922 - loss: 0.0256 - val_accuracy: 0.9814 - val_loss: 0.0643
Epoch 5/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.9946 - loss: 0.0184 - val_accuracy: 0.9790 - val_loss: 0.0739


In [8]:
# 6. Model GRU
def build_gru():
    model = Sequential([
        Embedding(10000, 64, input_length=100),
        Bidirectional(GRU(64)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

gru_model = build_gru()
gru_model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test), batch_size=64)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.9757 - loss: 0.1297 - val_accuracy: 0.9796 - val_loss: 0.0609
Epoch 2/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9844 - loss: 0.0470 - val_accuracy: 0.9818 - val_loss: 0.0632
Epoch 3/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.9902 - loss: 0.0322 - val_accuracy: 0.9817 - val_loss: 0.0625
Epoch 4/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 15s 21ms/step - accuracy: 0.9940 - loss: 0.0218 - val_accuracy: 0.9818 - val_loss: 0.0715
Epoch 5/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 17s 14ms/step - accuracy: 0.9955 - loss: 0.0162 - val_accuracy: 0.9781 - val_loss: 0.0734


In [15]:
# 7. Model IndoBERT
tokenizer_indo = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model_indo = TFAutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=2)

# Tokenization IndoBERT
def tokenize_function(example):
    # Explicitly set max_length and return as lists
    return tokenizer_indo(example["text"], padding="max_length", truncation=True, max_length=128)

dataset = Dataset.from_pandas(df[['cleaned', 'sentiment']].rename(columns={"cleaned": "text", "sentiment": "label"}))
dataset = dataset.train_test_split(test_size=0.2)
tokenized = dataset.map(tokenize_function, batched=True)

# Extract data as flat lists and convert to TensorFlow tensors
train_input_ids = tf.constant([item for sublist in tokenized["train"]["input_ids"] for item in sublist], dtype=tf.int32)
train_attention_mask = tf.constant([item for sublist in tokenized["train"]["attention_mask"] for item in sublist], dtype=tf.int32)
train_labels = tf.constant(tokenized["train"]["label"], dtype=tf.int32)

test_input_ids = tf.constant([item for sublist in tokenized["test"]["input_ids"] for item in sublist], dtype=tf.int32)
test_attention_mask = tf.constant([item for sublist in tokenized["test"]["attention_mask"] for item in sublist], dtype=tf.int32)
test_labels = tf.constant(tokenized["test"]["label"], dtype=tf.int32)

# Reshape the input_ids and attention_mask tensors
train_input_ids = tf.reshape(train_input_ids, (-1, 128))
train_attention_mask = tf.reshape(train_attention_mask, (-1, 128))
test_input_ids = tf.reshape(test_input_ids, (-1, 128))
test_attention_mask = tf.reshape(test_attention_mask, (-1, 128))


# Create TensorFlow datasets
features = tf.data.Dataset.from_tensor_slices(({'input_ids': train_input_ids, 'attention_mask': train_attention_mask}, train_labels)).shuffle(len(train_labels)).batch(16)
val_features = tf.data.Dataset.from_tensor_slices(({'input_ids': test_input_ids, 'attention_mask': test_attention_mask}, test_labels)).batch(16)


model_indo.compile(optimizer='adam', # Use string identifier for the optimizer
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

model_indo.fit(features, validation_data=val_features, epochs=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/31025 [00:00<?, ? examples/s]

Map:   0%|          | 0/7757 [00:00<?, ? examples/s]

Epoch 1/3
1940/1940 [==============================] - 888s 435ms/step - loss: 0.1272 - accuracy: 0.9748 - val_loss: 0.1176 - val_accuracy: 0.9754
Epoch 2/3
1940/1940 [==============================] - 858s 442ms/step - loss: 0.1230 - accuracy: 0.9763 - val_loss: 0.1163 - val_accuracy: 0.9754
Epoch 3/3
1940/1940 [==============================] - 844s 435ms/step - loss: 0.1255 - accuracy: 0.9763 - val_loss: 0.1210 - val_accuracy: 0.9754


In [16]:
# 8. Evaluasi
def evaluate_model(model, X, y_true, model_name="Model"):
    y_pred = (model.predict(X) > 0.5).astype("int32")
    print(f"\nEvaluation Report for {model_name}:\n")
    print(classification_report(y_true, y_pred))

evaluate_model(lstm_model, X_test, y_test, "LSTM")
evaluate_model(gru_model, X_test, y_test, "GRU")


243/243 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step

Evaluation Report for LSTM:

              precision    recall  f1-score   support

           0       0.56      0.56      0.56       186
           1       0.99      0.99      0.99      7571

    accuracy                           0.98      7757
   macro avg       0.78      0.77      0.77      7757
weighted avg       0.98      0.98      0.98      7757

243/243 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

Evaluation Report for GRU:

              precision    recall  f1-score   support

           0       0.54      0.58      0.56       186
           1       0.99      0.99      0.99      7571

    accuracy                           0.98      7757
   macro avg       0.76      0.78      0.77      7757
weighted avg       0.98      0.98      0.98      7757



In [17]:
from sklearn.metrics import classification_report
import numpy as np

y_true = []
y_pred = []

for batch in val_features:
    inputs, labels = batch
    logits = model_indo.predict(inputs)["logits"]
    y_pred.extend(np.argmax(logits, axis=-1))
    y_true.extend(labels.numpy())

print("Evaluation Report for IndoBERT:")
print(classification_report(y_true, y_pred))


1/1 [==============================] - 0s 123ms/step
Evaluation Report for IndoBERT:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       191
           1       0.98      1.00      0.99      7566

    accuracy                           0.98      7757
   macro avg       0.49      0.50      0.49      7757
weighted avg       0.95      0.98      0.96      7757



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
